**Book Recommendation Engine**
# Instructions
 In this challenge, you will create a book recommendation algorithm using K-Nearest Neighbors.You will use the Book-Crossings dataset. This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users.

 After importing and cleaning the data, use NearestNeighbors from sklearn.neighbors to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

 Create a function named get_recommends that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
 should return:

    [
      'The Queen of the Damned (Vampire Chronicles (Paperback))',
     [
      ['Catch 22', 0.793983519077301],
      ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479],
      ['Interview with the Vampire', 0.7345068454742432],
      ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
      ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
      ]
     ]
  

 Notice that the data returned from get_recommends() is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

 If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.


In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-01-24 12:44:27--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  28.2MB/s    in 0.9s    

2025-01-24 12:44:29 (28.2 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_books.head()


,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [5]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [6]:
df_books.isnull().sum()

,0
isbn,0
title,0
author,2


In [7]:
df_ratings.isnull().sum()

,0
user,0
isbn,0
rating,0


In [8]:
df_books.dropna(inplace=True)


In [9]:
df_books.isnull().sum()

,0
isbn,0
title,0
author,0


In [10]:
# Count the number of ratings for each user
user_rating_counts = df_ratings['user'].value_counts()
user_rating_counts.sort_values(ascending = False).head()

,count
user,
11676,13602
198711,7550
153662,6109
98391,5891
35859,5850


In [11]:
# Filter users who have rated at least 200 books
valid_users = user_rating_counts[user_rating_counts >= 200].index
df_ratings_fil = df_ratings[df_ratings['user'].isin(valid_users)]
df_ratings_fil.shape

(527556, 3)

In [12]:
# Calculate the number of ratings for each book (based on 'isbn')
book_rating_counts = df_ratings['isbn'].value_counts()
book_rating_counts.sort_values(ascending=False).head()

,count
isbn,
0971880107,2502
0316666343,1295
0385504209,883
0060928336,732
0312195516,723


In [13]:

# Filter books that have been rated at least 100 times
valid_books = book_rating_counts[book_rating_counts >= 100].index
df_ratings_fil = df_ratings_fil[df_ratings_fil['isbn'].isin(valid_books)]
df_ratings_fil.shape

(49781, 3)

In [14]:
# Create a user-item matrix where rows are users, columns are books, and values are ratings and transpose it.
user_item_matrix = df_ratings_fil.pivot_table(index=['user'], columns=['isbn'], values='rating').fillna(0).T
user_item_matrix.head()




user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Replace the ISBN-based index in the user-item matrix with book titles from df_books.
user_item_matrix.index = user_item_matrix.join(df_books.set_index('isbn'))['title']
user_item_matrix = user_item_matrix.sort_index()
user_item_matrix.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Check the shape of the user-item matrix
print(f"User-item matrix shape: {user_item_matrix.shape}")

User-item matrix shape: (731, 888)


In [17]:
title = 'The Queen of the Damned (Vampire Chronicles (Paperback))'
user_item_matrix.loc[title].shape

(888,)

In [18]:
# Initialize the KNN model
knn_model = NearestNeighbors(algorithm='brute', metric='cosine')


# Fit the model with the user-item matrix
knn_model.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [19]:
# unpack the diatance and indices
distance, indices = knn_model.kneighbors([user_item_matrix.loc[title].values], n_neighbors=6)

print(distance)
print(indices)

[[0.         0.51784116 0.53763384 0.73450685 0.74486566 0.7939835 ]]
[[612 660 648 272 667 110]]


In [20]:
# convert the title and distance into pandas DataFrame
pd.DataFrame({
    'title'   : user_item_matrix.iloc[indices[0]].index.values,
    'distance': distance[0]
}).sort_values(by='distance', ascending=False)

,title,distance
5,Catch 22,0.793984
4,The Witching Hour (Lives of the Mayfair Witches),0.744866
3,Interview with the Vampire,0.734507
2,The Tale of the Body Thief (Vampire Chronicles...,0.537634
1,"The Vampire Lestat (Vampire Chronicles, Book II)",0.517841
0,The Queen of the Damned (Vampire Chronicles (P...,0.000000


In [21]:
# Function to get the recommended books based on the input title
def get_recommends(title=""):

    if title not in user_item_matrix.index:
        print(f"Error: The book title '{title}' does not exist in the user-item matrix!")
        return

    try:

        book = user_item_matrix.loc[title]
    except KeyError as e:
        print(f"Error: Unable to find the book '{title}'. Error: {e}")
        return

    # Find the nearest neighbors
    distance, indices = knn_model.kneighbors([book], n_neighbors=6)

    # Prepare the recommended books dataframe
    recommended_books = pd.DataFrame({
        'title': user_item_matrix.iloc[indices[0]].index.values,
        'distance': distance[0]
    }).sort_values(by='distance', ascending=False).head(5)

    # Return the book title and the recommended books
    return [title, recommended_books.values]

In [23]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
